In [1]:
import re
import json
from pathlib import Path
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from scripts.speedlimit_only_dataset import load_xml, extract_roadsigns_annotation_data_from_xml

In [ ]:
roadsigns_annotations = []
for fp in Path("data/Road-Sign-Detection-v2/annotations").glob("*.xml"):
    roadsigns_annotations.append(extract_roadsigns_annotation_data_from_xml(load_xml(fp)[1]))

In [3]:
def make_coco_categories_list(labels):
    coco_categories_list = [{"id": idx, "name": x} for idx, x in enumerate(labels)]
    categories_mapper = {x["name"]: x["id"] for x in coco_categories_list}

    return coco_categories_list, categories_mapper


def make_coco_images_list_element(data):
    file_name = data["image_file"]
    image_width = int(data["image_width"])
    image_height = int(data["image_height"])
    image_id = re.match(r"\d+?_road(?P<digits>\d+?)\.png", file_name)
    return {"id": int(image_id.groupdict()["digits"]), "width": image_width, "height": image_height, "file_name": file_name, "date_captured": "2022-03-09 00:00:00"}


def make_coco_annotations_list_element_func():
    annotation_id = 0

    def wrapped(data, image_id, categories_mapper):
        nonlocal annotation_id

        for bounding_box in data["image_objects"]:
            category_id = int(categories_mapper[bounding_box["name"]])
            xmin = int(bounding_box["bndbox_xmin"])
            xmax = int(bounding_box["bndbox_xmax"])
            ymin = int(bounding_box["bndbox_ymin"])
            ymax = int(bounding_box["bndbox_ymax"])

            width = xmax - xmin
            height = ymax - ymin

            coco_annotation = {"id": annotation_id, "image_id": image_id, "category_id": category_id, "bbox": [xmin, ymin, width, height], "iscrowd": 0, "area": round(width * height, 1)}
            annotation_id += 1

            yield coco_annotation

    return wrapped

In [4]:
coco_categories_list, categories_mapper = make_coco_categories_list(["crosswalk", "speedlimit", "stop", "trafficlight"])

In [5]:
make_coco_annotations_list_element = make_coco_annotations_list_element_func()

coco_images_list = []
coco_annotations_list = []

for annotation in roadsigns_annotations:
    images_list_element = make_coco_images_list_element(annotation)
    coco_images_list.append(images_list_element)

    for annotation_list_element in make_coco_annotations_list_element(annotation, images_list_element["id"], categories_mapper):
        coco_annotations_list.append(annotation_list_element)

coco_images_list = sorted(coco_images_list, key=lambda x: x["id"])
coco_annotations_list = sorted(coco_annotations_list, key=lambda x: (x["image_id"], x["category_id"]))

In [6]:
coco_info = {"description": "Road Signs Dataset", "version": "2.0", "year": 2020}

In [7]:
coco_data = {
    "info": coco_info,
    "images": coco_images_list,
    "annotations": coco_annotations_list,
    "categories": coco_categories_list,
}

In [8]:
with Path("data/Road-Sign-Detection-v2/coco.json").open("wt") as f:
    json.dump(obj=coco_data, fp=f)

In [13]:
coco_gt = COCO(str(Path("data/Road-Sign-Detection-v2/coco.json")))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [14]:
coco_gt.anns

{343: {'id': 343, 'image_id': 0, 'category_id': 3, 'bbox': [97, 61, 73, 149]},
 367: {'id': 367,
  'image_id': 1,
  'category_id': 3,
  'bbox': [154, 62, 104, 218]},
 476: {'id': 476, 'image_id': 2, 'category_id': 3, 'bbox': [143, 270, 30, 82]},
 477: {'id': 477, 'image_id': 2, 'category_id': 3, 'bbox': [128, 148, 7, 16]},
 478: {'id': 478, 'image_id': 2, 'category_id': 3, 'bbox': [185, 139, 3, 9]},
 432: {'id': 432,
  'image_id': 3,
  'category_id': 3,
  'bbox': [178, 134, 57, 127]},
 663: {'id': 663, 'image_id': 4, 'category_id': 3, 'bbox': [20, 109, 61, 128]},
 664: {'id': 664,
  'image_id': 4,
  'category_id': 3,
  'bbox': [116, 162, 47, 110]},
 665: {'id': 665,
  'image_id': 4,
  'category_id': 3,
  'bbox': [189, 189, 43, 106]},
 628: {'id': 628, 'image_id': 5, 'category_id': 3, 'bbox': [46, 30, 174, 262]},
 555: {'id': 555, 'image_id': 6, 'category_id': 3, 'bbox': [95, 153, 81, 174]},
 585: {'id': 585, 'image_id': 7, 'category_id': 3, 'bbox': [120, 145, 39, 73]},
 586: {'id': 586